# Análise de Dados - Olist

Este jupyter notebook apresenta uma análise geral dos dados da plataforma de vendas Olist, organizando e visualizando informações sobre clientes, produtos e vendas.

### 1. Importação das Bibliotecas e Leitura dos Dados

Carregamos os arquivos CSV para os respectivos DataFrames.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

caminho_arquivos = {
    "clientes": "dados/olist_customers_dataset.csv",
    "geolocalizacao": "dados/olist_geolocation_dataset.csv",
    "itens_pedidos": "dados/olist_order_items_dataset.csv",
    "pagamentos_pedidos": "dados/olist_order_payments_dataset.csv",
    "avaliacoes_pedidos": "dados/olist_order_reviews_dataset.csv",
    "pedidos": "dados/olist_orders_dataset.csv",
    "produtos": "dados/olist_products_dataset.csv",
    "vendedores": "dados/olist_sellers_dataset.csv",
    "traducao_categorias_produtos": "dados/product_category_name_translation.csv",
}

dataframes = {nome: pd.read_csv(caminho) for nome, caminho in caminho_arquivos.items()}

clientes_df = dataframes["clientes"]
geolocalizacao_df = dataframes["geolocalizacao"]
itens_pedidos_df = dataframes["itens_pedidos"]
pagamentos_pedidos_df = dataframes["pagamentos_pedidos"]
avaliacoes_pedidos_df = dataframes["avaliacoes_pedidos"]
pedidos_df = dataframes["pedidos"]
produtos_df = dataframes["produtos"]
vendedores_df = dataframes["vendedores"]
traducao_categorias_produtos_df = dataframes["traducao_categorias_produtos"]

### 2. Pré-visualização dos Dados

Exibimos uma amostra de cada DataFrame para entender sua estrutura e conteúdo.

In [2]:
clientes_df.sample(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
32970,a94a658fc7f1752a7a94ad9eb11bcf38,27aacd37c12bc685a275af86f14d2c23,74663,goiania,GO
61022,27056b7474812bc0627c6490b5ab5a0b,214a7ab28ddecfeeca93b2a8550ba1aa,89258,jaragua do sul,SC


In [3]:
geolocalizacao_df.sample(2)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
616407,36400,-20.656258,-43.774131,conselheiro lafaiete,MG
594475,35500,-20.120165,-44.889216,divinopolis,MG


In [4]:
itens_pedidos_df.sample(2)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
12350,1c05452697a356ff98c4c3391e207ab6,2,1e7d94ec384eb160b91eb277f13ba1ff,8f2ce03f928b567e3d56181ae20ae952,2018-01-26 14:55:00,199.90,38.54
62770,8f50817faced9fa9f4c8807f6fc4548a,1,2b4609f8948be18874494203496bc318,cc419e0650a3c5ba77189a1882b7556a,2018-02-02 11:17:31,89.99,9.44


In [5]:
pagamentos_pedidos_df.sample(2)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
27438,f7c666a64887124db742dd9a8a4e300d,1,credit_card,4,57.72
45345,b1ce906877fd32ecc3216688454fe6cc,2,voucher,1,13.16


In [6]:
avaliacoes_pedidos_df.sample(2)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
82854,777b8de01824c1e8bf0a299a119d6424,477d8df8acc3c2b3f1fbb4e41c4f7f32,1,NaN,NaN,2018-07-06 00:00:00,2018-07-09 13:08:29
62967,ee49900281e153d00673d073350a67fe,dc69158eb223d2e98bbaaba4c9f81c87,1,NaN,Já foi cobrado no meu cartão de crédito mas at...,2018-02-17 00:00:00,2018-02-19 09:55:09


In [7]:
pedidos_df.sample(2)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
83963,8d6ed3e982e12a1684b31540309e044b,129468ef8647b08efc12e04af8506cfc,delivered,2017-09-13 15:10:21,2017-09-14 22:25:39,2017-09-19 20:19:44,2017-09-25 17:32:38,2017-09-29 00:00:00
2616,945cebd7f0d6886e7971663839507ec4,c69e9b9095b1491d57aed0483eb1b9e6,delivered,2017-12-06 13:07:22,2017-12-06 13:14:30,2017-12-07 18:09:14,2017-12-18 21:09:33,2017-12-29 00:00:00


In [8]:
produtos_df.sample(2)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
26083,f7c98cd8b2d4d3b9a2d1322721e60a5a,construcao_ferramentas_iluminacao,58.0,2025.0,1.0,3200.0,18.0,23.0,25.0
10344,f3f6c295f8683fd04c2a01c363db3ec5,livros_interesse_geral,34.0,486.0,1.0,600.0,23.0,7.0,16.0


In [9]:
vendedores_df.sample(5)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
2126,6bb62a4912377c7bfe7cb17b406bc7e6,13215,jundiai,SP
1728,98115075dd26cb8835946fc6086f5d30,9687,ao bernardo do campo,SP
1598,253f95b3a0e6983ae18aed49fbc2b845,4141,sao paulo,SP
363,34a557286ead8fe75c989e0afcf7d98c,14406,franca,SP
1383,817f85dbb65aa3e70831d90fe75cdf89,18530,tiete,SP


In [10]:
traducao_categorias_produtos_df.sample(2)

,product_category_name,product_category_name_english
23,malas_acessorios,luggage_accessories
33,eletrodomesticos,home_appliances


In [11]:
produtos_df.describe()

,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32341.000000,32341.000000,32341.000000,32949.000000,32949.000000,32949.000000,32949.000000
mean,48.476949,771.495285,2.188986,2276.472488,30.815078,16.937661,23.196728
std,10.245741,635.115225,1.736766,4282.038731,16.914458,13.637554,12.079047
min,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,42.000000,339.000000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,51.000000,595.000000,1.000000,700.000000,25.000000,13.000000,20.000000
75%,57.000000,972.000000,3.000000,1900.000000,38.000000,21.000000,30.000000
max,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000


### 3. Identificação de Valores Ausentes

Identificamos valores ausentes em cada coluna dos DataFrames e organizamos os resultados em uma tabela consolidada para melhor visualização.

In [12]:
resumo = []

for nome, df in dataframes.items():
    valores_ausentes = df.isnull().sum() 
    df_resumo = pd.DataFrame({
        "DataFrame": [nome] * len(valores_ausentes),
        "Coluna": valores_ausentes.index,
        "Valores ausentes": valores_ausentes.values
    })
    resumo.append(df_resumo)
    
df_resumo_final = pd.concat(resumo, ignore_index=True)
df_resumo_final


,DataFrame,Coluna,Valores ausentes
0,clientes,customer_id,0
1,clientes,customer_unique_id,0
2,clientes,customer_zip_code_prefix,0
3,clientes,customer_city,0
4,clientes,customer_state,0
5,geolocalizacao,geolocation_zip_code_prefix,0
6,geolocalizacao,geolocation_lat,0
7,geolocalizacao,geolocation_lng,0
8,geolocalizacao,geolocation_city,0
9,geolocalizacao,geolocation_state,0


### 4. Tratamento de Dados

#### 4.1 Substituição de Valores Ausentes

In [13]:
avaliacoes_pedidos_df['review_comment_message'] = avaliacoes_pedidos_df['review_comment_message'].fillna('Sem Comentário')
avaliacoes_pedidos_df['review_comment_title'] = avaliacoes_pedidos_df['review_comment_title'].fillna('Sem Título')

In [14]:
# preenche valores ausentes com a média nas colunas numéricas do DataFrame 'produtos_df'
colunas_numericas = ['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']

produtos_df[colunas_numericas] = produtos_df[colunas_numericas].apply(lambda col: col.fillna(col.mean()))

### 5. Análise Exploratória de Dados (EDA)

Exploramos os dados para obter _insights_ sobre tempos de entrega, vendas por categoria e vendas por localização geográfica.

In [15]:
pedidos_df['order_delivered_customer_date'] = pd.to_datetime(pedidos_df['order_delivered_customer_date'])
pedidos_df['order_purchase_timestamp'] = pd.to_datetime(pedidos_df['order_purchase_timestamp'])
pedidos_df['delivery_time'] = (pedidos_df['order_delivered_customer_date'] - pedidos_df['order_purchase_timestamp']).dt.days

tempo_medio_entrega = pedidos_df['delivery_time'].mean()
tempo_medio_entrega = round(tempo_medio_entrega, 2)
print(f"Tempo médio de entrega: {tempo_medio_entrega} dias")


Tempo médio de entrega: 12.09 dias


In [16]:
itens_produtos = pd.merge(itens_pedidos_df, produtos_df, on='product_id', how='inner')

vendas_por_categoria = itens_produtos['product_category_name'].value_counts()

vendas_por_categoria_df = pd.DataFrame(vendas_por_categoria).reset_index()
vendas_por_categoria_df.columns = ['Categoria de Produto', 'Frequência de Vendas']

vendas_por_categoria_df


,Categoria de Produto,Frequência de Vendas
0,cama_mesa_banho,11115
1,beleza_saude,9670
2,esporte_lazer,8641
3,moveis_decoracao,8334
4,informatica_acessorios,7827
...,...,...
68,cds_dvds_musicais,14
69,la_cuisine,14
70,pc_gamer,9
71,fashion_roupa_infanto_juvenil,8


In [17]:
vendas_geolocalizacao = pd.merge(clientes_df, geolocalizacao_df, 
                                  left_on='customer_zip_code_prefix', 
                                  right_on='geolocation_zip_code_prefix', 
                                  how='inner')

vendas_por_estado = vendas_geolocalizacao['geolocation_state'].value_counts()

vendas_por_estado_df = pd.DataFrame(vendas_por_estado).reset_index()
vendas_por_estado_df.columns = ['Estado', 'Frequência de Vendas']

vendas_por_estado_df


,Estado,Frequência de Vendas
0,SP,5620430
1,RJ,3015690
2,MG,2878728
3,RS,805370
4,PR,626021
5,SC,538638
6,BA,365875
7,ES,316654
8,GO,133146
9,MT,122395


In [18]:
vendas_por_cidade = vendas_geolocalizacao['geolocation_city'].value_counts()

vendas_por_cidade_df = pd.DataFrame(vendas_por_cidade).reset_index()
vendas_por_cidade_df.columns = ['Cidade', 'Frequência de Vendas']

vendas_por_cidade_df


,Cidade,Frequência de Vendas
0,rio de janeiro,1913913
1,sao paulo,1164470
2,belo horizonte,737556
3,niteroi,393175
4,curitiba,255731
...,...,...
5807,josé boiteux,1
5808,pinhal da serra,1
5809,vitorinos,1
5810,vitorinos - alto rio doce,1
